In [ ]:
# Import required libraries and dependencies
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame and make the index the "coin_id" column.
market_data_df = pd.read_csv("Resources/crypto_market_data.csv", index_col="coin_id")

# Display sample data
market_data_df.head(10)

In [ ]:
# Generate summary statistics
market_data_df.describe()

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler()
#normalize data
normalized_data = scaler.fit_transform(market_data_df)

In [ ]:
# Create a DataFrame with the scaled data
# Copy the crypto names from the original data
# Set the coinid column as index

scaled_df = pd.DataFrame(normalized_data,columns=market_data_df.columns,index=market_data_df.index)



# Display sample data
scaled_df.head()


### Find the Best Value for k Using the Original Scaled DataFrame.

In [ ]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k_values = range(1,12)

# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
for k in k_values:
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
    kmean = KMeans(n_clusters=k,random_state=0)
# 2. Fit the model to the data using the scaled DataFrame
    kmean.fit(scaled_df)
# 3. Append the model.inertia_ to the inertia list
    inertia.append(kmean.inertia_)


# Create a dictionary with the data to plot the Elbow curve
elbowCurve = {'k':k_values, 'inertia':inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbowDf = pd.DataFrame(elbowCurve)

# Display the DataFrame
elbowDf

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbowDf.plot(x='k',y='inertia',kind='line',marker="o")

#### Answer the following question: 
**Question:** What is the best value for `k`?

**Answer:** The best vaule for k would be 4

### Cluster Cryptocurrencies with K-means Using the Original Scaled Data.

In [ ]:
# Initialize the K-Means model using the best value for k
k_value = 4
k_means = KMeans(n_clusters=k_value,random_state=0)


In [ ]:
# Fit the K-Means model using the scaled data
k_means.fit(scaled_df)


In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters = k_means.predict(scaled_df)

# View the resulting array of cluster values.
clusters

In [ ]:
# Create a copy of the DataFrame
cluster_df = scaled_df.copy()

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
cluster_df['Clusters'] = clusters

# Display sample data
cluster_df.head()

In [ ]:
# Create a scatter plot using Pandas plot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Use "rainbow" for the color to better visualize the data.
cluster_df.plot.scatter(x='price_change_percentage_24h',y="price_change_percentage_7d",c='Clusters',colormap="rainbow")

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` on the original scaled DataFrame to reduce to three principal components.
pca_model = pca.fit_transform(scaled_df)
pca_df = pd.DataFrame(data=pca_model, columns=["PC1","PC2","PC3"], index=scaled_df.index)

# View the first five rows of the DataFrame. 
pca_df.head()

In [ ]:
# Retrieve the explained variance to determine how much information  can be attributed to each principal component.
ev = pca.explained_variance_ratio_
total_ev = ev.sum()
#Display explained variance and total explained variance
ev,total_ev

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 89.5%

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values to try
# Use a range from 1 to 11
k_values_pca = range(1,12)

# Create an empty list to store the inertia values
inertia_values_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop: 
for k in k_values_pca:
    # 1. Create a KMeans model using the loop counter for the n_clusters
    kmeans_pca = KMeans(n_clusters=k,random_state=0)
    # 2. Fit the model to the data using PCA DataFrame.
    kmeans_pca.fit(pca_df)
    #3. Append the model.inertia_ to the inertia list
    inertia_values_pca.append(kmeans_pca.inertia_)

# Create a dictionary with the data to plot the Elbow curve
# Create a DataFrame with the data to plot the Elbow curve
elbow_df_pca = pd.DataFrame({'k': k_values_pca , 'inertia': inertia_values_pca})

# Display the DataFrame
elbow_df_pca


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_df_pca.plot(x='k',y='inertia',kind='line',marker="o")

#### Answer the following questions: 
* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**  The best value is 4


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:**  No it does not differ

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
best_k = 4
kmeans_pca = KMeans(n_clusters=best_k, random_state=0)


In [ ]:
# Fit the K-Means model using the PCA data
kmeans_pca.fit(pca_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
clusters_pca = kmeans_pca.predict(pca_df)

# Print the resulting array of cluster values.
clusters_pca

In [ ]:
# Create a copy of the DataFrame with the PCA data
pca_clustered_df = pca_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_clustered_df["Cluster"] = clusters_pca

# Display sample data
pca_clustered_df.head()

In [ ]:
# Create a scatter plot using hvPlot by setting `x="PCA1"` and `y="PCA2"`. 
pca_clustered_df.plot.scatter(x="PC1",y='PC2',c="Cluster",colormap="rainbow")

### Determine the Weights of Each Feature on each Principal Component

In [ ]:
# Use the columns from the original scaled DataFrame as the index.
weights = pd.DataFrame(pca.components_.T,columns=["PC1","PC2","PC3"],index=scaled_df.columns)
#Display weights
weights

#### Answer the following question: 

* **Question:** Which features have the strongest positive or negative influence on each component? 
 
* **Answer:** Price change percentage 200d and 1y have most positive influence on the PC1 while 24h and 7d have the strongest negative influence.  PC2 is dominated by 14d and 30d while has the most negative influence from 1y. Lastly PC3 is dominated most by 7d while has the most negative influence from 60d and 24h.
    